In [6]:
import pandas as pd
df = pd.read_csv("data/WorkingData.csv").set_index("ContestId")
df.head()

,SportName,VariantName,GameSet,ContestName,ContestStartDatetimeEST,ContestEndDatetimeEST,ContestPayoutDatetimeEST,EntryFeeAmount,TotalPrizeAmount,MaxNumberPlayers,MaxEntriesPerUser,Entries,DistinctUsers,Contest_Group,NumGames,DraftablePlayersInSet,PaidUsersInDraftGroup,TopPrize,MaxPayoutPosition,is_vault
ContestId,,,,,,,,,,,,,,,,,,,,
7963004,NFL,Classic,(Main),NFL $25K Quarter Arcade [Just $0.25!],2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,0.25,25000.0,117500,100,117500.0,38925,Headliner,15,910.0,668396,2000.0,23665,False
7963437,NFL,Classic,(Main),"NFL GIANT $2 Booster [Top 1,250 Win $20]",2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,2.00,25000.0,14375,100,14375.0,9663,Booster,15,910.0,668396,20.0,1250,False
8296224,NFL,Classic,(Main),Beginner NFL $5K Spy [Single Entry],2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,100.00,5000.0,55,1,55.0,55,SingleEntry,15,910.0,668396,1100.0,11,False
8296544,NFL,Classic,(Main),NFL $50 Double Up [$5K Gtd] (Multi-Entry),2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,50.00,5000.0,111,11,111.0,91,FeaturedDoubleUp,15,910.0,668396,100.0,50,False
8494213,NFL,Classic,(Main),"NFL $300K Flea Flicker [$300,000 Guaranteed]",2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,5.00,300000.0,69000,200,57604.0,38743,Headliner,15,910.0,668396,25000.0,13805,False
